In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from PIL import Image
import random

import os
import json
import cv2

import torch
from torch.utils.data import random_split, DataLoader, Dataset
from torchvision import datasets, transforms

from transformers import TrOCRProcessor, VisionEncoderDecoderModel, default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import albumentations as A



Завиксируем всю случайность!

In [2]:
seed = 23
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.determenistic = True

Соберем наш датасет из полученных файлов

In [3]:
basketball_df = pd.read_csv('for_train\\train_bascetball.csv')
streetball_df = pd.read_csv('for_train\\train_streetball.csv')
volleyball_df = pd.read_csv('for_train\\train_volleyball.csv')

res_df = pd.concat([basketball_df, streetball_df, volleyball_df], axis=0)
res_df.shape

(150637, 3)

Разобьем датасет на train test и eval части

In [4]:
diff_df, test_df, = train_test_split(res_df, test_size=0.2, shuffle=True)
train_df, eval_df = train_test_split(diff_df, test_size=0.2)
# we reset the indices to start from zero
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
eval_df.reset_index(drop=True, inplace=True)

In [5]:
transform = A.Compose([
    A.Perspective(),
    A.Rotate(45),
    A.Resize(height=40, width=40),
    A.RandomCrop(width=32, height=32)   
])

In [6]:
class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=3, transforms = None):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
      
        file_name = self.df['file_name'][idx]
        text = str(self.df['text'][idx])
   
        image = Image.open(file_name).convert("RGB")
        
        if self.transforms is not None:
            image = self.transforms(image=np.array(image))["image"]
        
        #image = image.resize((64, 64))
        pixel_values = self.processor(image, return_tensors="pt").pixel_values
        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(text, 
                                          padding="max_length", 
                                          max_length=self.max_target_length).input_ids
        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

Создадим наши датасеты

In [7]:
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
train_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=train_df,
                           processor=processor,
                           transforms=transform)
test_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=test_df,
                           processor=processor)
eval_dataset = IAMDataset(root_dir='C:\\Users\\Mytre\\OneDrive\\Документы\\Data\\Work\\',
                           df=eval_df,
                           processor=processor)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
print("Number of training examples:", len(train_dataset))
print("Number of testing examples:", len(test_dataset))
print("Number of validation examples:", len(eval_dataset))

Number of training examples: 96407
Number of testing examples: 30128
Number of validation examples: 24102


Загрузим предъобученный трансформер

In [9]:
#model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-stage1")
model = VisionEncoderDecoderModel.from_pretrained("model")

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#model = VisionEncoderDecoderModel.from_pretrained('model')
model.to(device)
print(device)

cuda


Сконфигурируем нашу модель

In [11]:
# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Сконфигурируем цикл обучения

In [12]:
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    num_train_epochs=40,
    evaluation_strategy="steps",
    per_device_train_batch_size=3,
    per_device_eval_batch_size=3,
    fp16=True, 
    output_dir="G:\\models1",
    logging_steps=1000,
    save_steps=10000,
    eval_steps=10000,
)

Оопределим метрику

In [13]:
import evaluate

cer_metric = evaluate.load("cer")
acc_metric = evaluate.load("accuracy")

In [14]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions    

    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(labels_ids, skip_special_tokens=True)    

    label_int = [int(x) for x in label_str]
    x = [] 
    for j in pred_str:       
        if j.isdigit():
            x.append(int(j))
        else:
            x.append(1000)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    acc = acc_metric.compute(predictions=x, references=label_int)

    return {"cer": cer,
            "accuracy": acc}


Переопределим оптимизатор

In [15]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),lr=0.00001)

Запустим цикл обучения

In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=processor.feature_extractor,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
)
trainer.train()

model.save_pretrained('model_next')

c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\models\trocr\processing_trocr.py:134: FutureWarning: `feature_extractor` is deprecated and will be removed in v5. Use `image_processor` instead.
  warnings.warn(
c:\Users\Mytre\OneDrive\Документы\Data\Work\.env\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1285440 [00:00<?, ?it/s]

{'loss': 0.3006, 'learning_rate': 4.9961336196166296e-05, 'epoch': 0.03}
{'loss': 0.2464, 'learning_rate': 4.992243900921086e-05, 'epoch': 0.06}
{'loss': 0.2321, 'learning_rate': 4.988354182225542e-05, 'epoch': 0.09}
{'loss': 0.2251, 'learning_rate': 4.984464463529997e-05, 'epoch': 0.12}
{'loss': 0.1998, 'learning_rate': 4.980574744834454e-05, 'epoch': 0.16}
{'loss': 0.2131, 'learning_rate': 4.97668502613891e-05, 'epoch': 0.19}
{'loss': 0.2032, 'learning_rate': 4.9727991971620615e-05, 'epoch': 0.22}
{'loss': 0.2137, 'learning_rate': 4.9689094784665175e-05, 'epoch': 0.25}
{'loss': 0.1983, 'learning_rate': 4.9650197597709734e-05, 'epoch': 0.28}
{'loss': 0.2073, 'learning_rate': 4.961137820512821e-05, 'epoch': 0.31}


  0%|          | 0/8034 [00:00<?, ?it/s]

{'eval_loss': 4.424245834350586, 'eval_cer': 0.1932088285229202, 'eval_accuracy': {'accuracy': 0.7943739108787653}, 'eval_runtime': 4547.2131, 'eval_samples_per_second': 5.3, 'eval_steps_per_second': 1.767, 'epoch': 0.31}
{'loss': 0.178, 'learning_rate': 4.957248101817277e-05, 'epoch': 0.34}
{'loss': 0.2035, 'learning_rate': 4.953358383121733e-05, 'epoch': 0.37}
{'loss': 0.1783, 'learning_rate': 4.9494686644261887e-05, 'epoch': 0.4}
{'loss': 0.1939, 'learning_rate': 4.945586725168036e-05, 'epoch': 0.44}
{'loss': 0.1896, 'learning_rate': 4.941697006472492e-05, 'epoch': 0.47}
{'loss': 0.1837, 'learning_rate': 4.937807287776948e-05, 'epoch': 0.5}
{'loss': 0.1886, 'learning_rate': 4.9339175690814046e-05, 'epoch': 0.53}
{'loss': 0.1926, 'learning_rate': 4.9300278503858605e-05, 'epoch': 0.56}
{'loss': 0.1801, 'learning_rate': 4.926142021409012e-05, 'epoch': 0.59}
{'loss': 0.1848, 'learning_rate': 4.922252302713468e-05, 'epoch': 0.62}


  0%|          | 0/8034 [00:00<?, ?it/s]

{'eval_loss': 4.445227146148682, 'eval_cer': 0.1832937181663837, 'eval_accuracy': {'accuracy': 0.7944154012115178}, 'eval_runtime': 4354.262, 'eval_samples_per_second': 5.535, 'eval_steps_per_second': 1.845, 'epoch': 0.62}
{'loss': 0.1788, 'learning_rate': 4.918362584017924e-05, 'epoch': 0.65}
{'loss': 0.1806, 'learning_rate': 4.914476755041076e-05, 'epoch': 0.68}
{'loss': 0.1766, 'learning_rate': 4.9105909260642275e-05, 'epoch': 0.72}
{'loss': 0.1666, 'learning_rate': 4.9067012073686834e-05, 'epoch': 0.75}
{'loss': 0.1575, 'learning_rate': 4.902815378391835e-05, 'epoch': 0.78}
{'loss': 0.1908, 'learning_rate': 4.898929549414987e-05, 'epoch': 0.81}
{'loss': 0.1885, 'learning_rate': 4.895039830719443e-05, 'epoch': 0.84}
{'loss': 0.1699, 'learning_rate': 4.891150112023899e-05, 'epoch': 0.87}
{'loss': 0.1918, 'learning_rate': 4.8872603933283546e-05, 'epoch': 0.9}
{'loss': 0.1843, 'learning_rate': 4.8833706746328105e-05, 'epoch': 0.93}


  0%|          | 0/8034 [00:00<?, ?it/s]

{'eval_loss': 4.519834518432617, 'eval_cer': 0.6140237691001698, 'eval_accuracy': {'accuracy': 0.366235167206041}, 'eval_runtime': 4455.1351, 'eval_samples_per_second': 5.41, 'eval_steps_per_second': 1.803, 'epoch': 0.93}


KeyboardInterrupt: 

In [ ]:
url = "crops\\8\\ballerTV_137example.jpg"
image = Image.open(url).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values.cuda())

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(f'Номер на футболке: {generated_text}')

Определим eval_dataloader и Загрузим обученную модель

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=32)
model = VisionEncoderDecoderModel.from_pretrained('G:\\models1\\checkpoint-40000')

# set special tokens used for creating the decoder_input_ids from the labels
model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size

# set beam search parameters
model.config.eos_token_id = processor.tokenizer.sep_token_id
model.config.max_length = 4
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 10

Посчитаем точность обученной модели на eval датасете

In [ ]:
from sklearn.metrics import accuracy_score

torch.cuda.empty_cache()
# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
diff_acc = []
acc = []
i = 0

with torch.no_grad():
    for batch in tqdm(test_dataloader):
    
        target_text = processor.batch_decode(batch['labels'], skip_special_tokens=True)
        target_text = [int(x) for x in target_text]
           
        pixel_values = batch['pixel_values']
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=4)       
       
        # Make a prediction
        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)

        x = [] 
        for j in generated_text:       
            if j.isdigit():
                x.append(int(j))
            else:
                x.append(1000)        
      
        bach_acc = accuracy_score(target_text, x)
        # Save the true and predicted labels
        diff_acc.append(bach_acc)  
        acc.append(bach_acc)

        if (i % 50  == 0) & (i > 2):
            accuracy = np.mean(diff_acc)
            print("Accuracy:", accuracy) 
        i += 1

    print(f"Total accuracy: {np.mean(acc)}")    

  0%|          | 0/942 [00:00<?, ?it/s]

Accuracy: 0.8768382352941176
Accuracy: 0.880569306930693
Accuracy: 0.8849337748344371
Accuracy: 0.8851057213930348
Accuracy: 0.8813496015936255
Accuracy: 0.8822674418604651
Accuracy: 0.8839921652421653
Accuracy: 0.8827150872817955
Accuracy: 0.8817904656319291
Accuracy: 0.8832959081836327
Accuracy: 0.8844714156079855
Accuracy: 0.8843074043261231
Accuracy: 0.8849366359447005
Accuracy: 0.8859218972895863
Accuracy: 0.8852363515312917
Accuracy: 0.8845583645443196
Accuracy: 0.8843639835487661
Accuracy: 0.884225860155383
Total accuracy: 0.8842887473460722


Посчитаем точность обученной модели на eval датасете

In [ ]:
from sklearn.metrics import accuracy_score

# Define the device to run the evaluation on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the model to evaluation mode
model.eval()
model.to(device)
# Evaluate the model on the eval dataset
y_true = []
y_pred = []
xx = []
with torch.no_grad():
    for i in tqdm(range(len(test_dataset)), nrows=2):
        url = test_df['file_name'][i]
        xx.append(url)
        target_text = int(test_df['text'][i])
        image = Image.open(url).convert("RGB")   

        pixel_values = processor(image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values.cuda(), max_new_tokens=3)

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        #pixel_values = batch['pixel_values']
        #generated_ids = model.generate(pixel_values.cuda())       

        # Make a prediction
        #generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
        x = []        
        if generated_text.isdigit():
            
            x = int(generated_text)
        else:
            x = 1000    
        
        # Save the true and predicted labels
        y_true.append(target_text)
        y_pred.append(x)

        if (i % 400  == 0) & (i > 2):
            accuracy = accuracy_score(y_true, y_pred)
            print("Accuracy:", accuracy) 

print("Accuracy:", accuracy)                   